In [1]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import requests
import re
import time
import pandas as pd

In [35]:
#URLs

nasa_news_url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
jpl_images_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
mars_weather_twitter = 'https://twitter.com/marswxreport?lang=en'
mars_facts = 'https://space-facts.com/mars/'
USGS = 'https://web.archive.org/web/20181114171728/https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

In [3]:
#Mars News

response = requests.get(nasa_news_url)
soup = bs(response.text, 'lxml')

news_titles = soup.find_all('div', class_='content_title')
blurbs = soup.find_all('div', class_='rollover_description_inner')

news_title = news_titles[0].get_text(strip='True')
blurb = blurbs[0].get_text(strip='True')

print(news_title)
print(blurb)

NASA Invites Students to Name Mars 2020 Rover
Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.


In [70]:
#JPL Mars Space Images - Featured Image

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

browser.visit(jpl_images_url)
browser.click_link_by_partial_text('FULL IMAGE')

time.sleep(5)

html = browser.html
soup = bs(html, 'html.parser')
featured_image = soup.find('img', class_= 'fancybox-image')

time.sleep(5)

In [72]:
img_source = featured_image['src']
img_name = img_source.split('/')[4]

img_name_hires = img_name.split('_')[0] + "_hires.jpg"

print(img_name_hires)
JPL_image_src = 'https://www.jpl.nasa.gov/spaceimages/images/largesize/' + img_name_hires
    
print(JPL_image_src)

PIA22949_hires.jpg
https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA22949_hires.jpg


In [73]:
#Mars Weather from latest tweet

response = requests.get(mars_weather_twitter)
soup = bs(response.text, 'lxml')

tweets = soup.find('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text')

tweets.a.decompose()
latest_weather = tweets.get_text()

print(latest_weather)

InSight sol 359 (2019-11-30) low -99.3ºC (-146.8ºF) high -21.8ºC (-7.2ºF)
winds from the SSE at 5.8 m/s (13.0 mph) gusting to 23.3 m/s (52.0 mph)
pressure at 6.70 hPa


In [74]:
# Mars Facts

tables = pd.read_html(mars_facts)
df = tables[0]
df.columns = ['Category', 'Value']

In [75]:
html_table = df.to_html(index=False)
html_table.replace('\n', '')
print(html_table)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Category</th>
      <th>Value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <td>Recorded By:</td>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


In [76]:
mars_facts_html = open('mars_facts.html','w')
mars_facts_html.write(html_table)

917

In [69]:
#Mars Hemispheres

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

hemisphere_image_urls = [
    {"title": "Valles Marineris Hemisphere", "img_url": "..."},
    {"title": "Cerberus Hemisphere", "img_url": "..."},
    {"title": "Schiaparelli Hemisphere", "img_url": "..."},
    {"title": "Syrtis Major Hemisphere", "img_url": "..."},
]

browser.visit(USGS)
html = browser.html
soup = bs(html, 'html.parser')
time.sleep(2)

for x in range (0,4):

    link_text = hemisphere_image_urls[x]['title']
    browser.click_link_by_partial_text(link_text)
    time.sleep(3)
    link_ref = browser.find_link_by_text('Sample')['href']    
    hemisphere_image_urls[x]['img_url'] = link_ref 
    print(hemisphere_image_urls[x]['img_url'])
    browser.back()
    time.sleep(2)



https://web.archive.org/web/20181114182248/http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg
https://web.archive.org/web/20181114182238/http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
https://web.archive.org/web/20181114182242/http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
https://web.archive.org/web/20181114182245/http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
